# Grab the FGS count rate, RA, and Dec for the Guide Star in the Guider being used

First import the `fgscountrate` tool which will enable use to get the FGS count rate, RA, and Dec of the guide star, and any other information that comes from the Guide Star Catalog.

In [3]:
import numpy as np

import fgscountrate

Give the Guide Star ID and the guider being used for this observation

In [ ]:
guide_star_id = 'S4OL071716' # Replace this with the actual Guide Star ID for this observation
guider_number = 1 # Replace this with the guider number used for this observation

## Get the Count Rate 

Run the following cells to get the FGS count rate of this guide star

In [ ]:
fgs = fgscountrate.FGSCountrate(guide_star_id=guide_star_id, guider=guider_number) 
count_rate_242, _, magnitude_242, _ = fgs.query_fgs_countrate_magnitude(catalog='GSC242')

In [ ]:
print(count_rate_242)

In [ ]:
print(magnitude_242)

## Compare with GSC 2.3.4

Given some inconsistencies seen in GA obs1, we now are checking to make sure that the 2MASS bands were not overwritten between GSC 2.3.4 and GSC 2.4.2. Here we check if the two catalogs give the same or different answers. 

*If the answers are different, **use the GSC 2.3.4** magnitudes*

In [ ]:
fgs = fgscountrate.FGSCountrate(guide_star_id=guide_star_id, guider=guider_number) 
count_rate_23, _, magnitude_23, _ = fgs.query_fgs_countrate_magnitude(catalog='GSC23')

In [ ]:
print(count_rate_23)

In [ ]:
print(magnitude_23)

In [ ]:
if not np.isclose(magnitude_242, magnitude_23):
    print(f'Mismatch in GSC 2.4.2 and GSC 2.3! Use the GSC 2.3 magnitude: {magnitude_23}')

## Get the RA and Dec

Run the following cells to get the RA and Dec. Note that the data frame that is passed back has a lot of information that might be helpful to you. 

In [ ]:
data_frame = fgscountrate.utils.query_gsc(gs_id=guide_star_id)

In [ ]:
data_frame

In [ ]:
ra = data_frame['ra'].values[0]
dec = data_frame['dec'].values[0]

In [ ]:
print(f"RA: {ra}")
print(f"Dec: {dec}")

## Compare Catalog Values for a List of GS IDs

For the case where we are stacked and will be running guiding with multiple possible guide stars, we may want to test a large list of guide star IDs to check their discrepancies between Guide Star Catalogs.

From discussions with the FGS SI team, we decided that an acceptable cut off for useable guide star IDs were ones where the percent difference between the new and old catalog was not greater than 50%. This is not a perfect system, but we have seen success with 50% changes in the past. 

Below is code that will check and print out all the stars with some discrepancy and then all the stars that do not meet this 50% requirement

In [4]:
# Paste the GSIDs from the Guide Star Report in between the sets of quotes
# Seperate the IDs by spaces/enters (remove any commas/non-ID words) and the code will figure it out

guide_star_id_list ="""

S1HP000309   S1HP082727  S1HP083237  S1HP083463  S1HP627502  S1HP627527
S1HP082727    S1HP000309  S1HP082389  S1HP627502  S1HP627527  S1HP699282
S1HP082311    S1HP081346  S1HP082130  S1HP627651  S1HP699293
S1HP735867   S1HP088978  S1HP736126
S1HP078292 
S1HP078629  

S1HP620356   
S1HP085881    S1HP086094  S1HP627328
S1HP086094    S1HP085881  S1HP627351
S1HP084514  
S1HP082254  S1HP082573  S1HP738376

""".split()
guide_star_id_list = np.unique(guide_star_id_list)

print(f"The total number of unique guide star IDs in this visit file is {len(guide_star_id_list)}")

The total number of unique guide star IDs in this visit file is 27


In [5]:
print('Printing all differences in ID catalog values > 0.1 or IDs that cannot be checked')
for guider_number in [1,2]:
    print(f'For Guider {guider_number}')
    for guide_star_id in guide_star_id_list:
        try:
            fgs = fgscountrate.FGSCountrate(guide_star_id=guide_star_id, guider=guider_number) 
            count_rate_242, _, magnitude_242, _ = fgs.query_fgs_countrate_magnitude(catalog='GSC242')

            fgs = fgscountrate.FGSCountrate(guide_star_id=guide_star_id, guider=guider_number) 
            count_rate_23, _, magnitude_23, _ = fgs.query_fgs_countrate_magnitude(catalog='GSC23')
        except NameError as err:
            print(f'  GS ID: {guide_star_id} Error is {str(err)}')
            continue

        del fgs

        if np.abs(magnitude_242 - magnitude_23) > 0.1:
            print(f'  GS ID: {guide_star_id} has a difference of {np.abs(magnitude_242-magnitude_23)} magnitudes')

Printing all differences in ID catalog values > 0.1 or IDs that cannot be checked
For Guider 1
  GS ID: S1HP000309 has a difference of 0.13608408619410817 magnitudes
  GS ID: S1HP078292 has a difference of 0.10185501915592177 magnitudes
  GS ID: S1HP078629 has a difference of 0.10266664066728381 magnitudes
  GS ID: S1HP082389 has a difference of 0.10436240080485604 magnitudes
  GS ID: S1HP082573 has a difference of 0.101002588711534 magnitudes
  GS ID: S1HP082727 has a difference of 0.11519336769069 magnitudes
  GS ID: S1HP083237 has a difference of 0.1556625018529143 magnitudes
  GS ID: S1HP083463 has a difference of 0.10237993053448058 magnitudes
  GS ID: S1HP084514 has a difference of 0.11944099741995018 magnitudes
  GS ID: S1HP085881 has a difference of 0.16996259387309998 magnitudes
  GS ID: S1HP620356 Error is No guide stars match these requirements in catalog GSC23
  GS ID: S1HP627328 Error is No guide stars match these requirements in catalog GSC23
  GS ID: S1HP627351 Error is 

In [ ]:
print('Printing all differences in ID catalog values > 50%')
for guider_number in [1,2]:
    print(f'For Guider {guider_number}')
    for guide_star_id in guide_star_id_list:
        try:
            fgs = fgscountrate.FGSCountrate(guide_star_id=guide_star_id, guider=guider_number) 
            count_rate_242, _, magnitude_242, _ = fgs.query_fgs_countrate_magnitude(catalog='GSC242')

            fgs = fgscountrate.FGSCountrate(guide_star_id=guide_star_id, guider=guider_number) 
            count_rate_23, _, magnitude_23, _ = fgs.query_fgs_countrate_magnitude(catalog='GSC23')
        except NameError as err:
            continue

        del fgs

        if np.abs((count_rate_242 - count_rate_23)/count_rate_23) > 0.5:
            print(f' GS ID: {guide_star_id} has a % difference of {round((count_rate_242 - count_rate_23)/count_rate_23 * 100)}%')

Printing all differences in ID catalog values > 50%
For Guider 1


## About this notebook
Author: K. Brooks
Last updated: 19 February, 2022